# Comprehensive Database Schema Design

This notebook provides the complete, optimized database schema for the HomeVision AI platform.

## Goals:
1. Handle all floor types (basement, main, second, third, attic)
2. Support multi-floor homes
3. Enable image-to-room tagging
4. Track spatial relationships
5. Support digital twin visualization
6. Enable AI analysis tracking

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("✅ Libraries imported successfully")

## 1. Complete Schema Overview

### Entity Relationship:
```
User (1) ──→ (N) Home
Home (1) ──→ (N) Floor
Floor (1) ──→ (1) FloorPlan
Floor (1) ──→ (N) Room
Room (1) ──→ (N) RoomImage
Room (1) ──→ (N) Material
Room (1) ──→ (N) Fixture
Room (1) ──→ (1) SpatialData
```

In [ ]:
# Complete schema definition
complete_schema = {
    "homes": {
        "table_name": "homes",
        "description": "Core table for home/property information",
        "columns": [
            {"name": "id", "type": "UUID", "constraints": "PRIMARY KEY", "description": "Unique identifier"},
            {"name": "owner_id", "type": "UUID", "constraints": "FOREIGN KEY (users.id)", "description": "Owner reference"},
            {"name": "name", "type": "VARCHAR(255)", "constraints": "NOT NULL", "description": "Home name/nickname"},
            {"name": "address", "type": "JSONB", "constraints": "NOT NULL", "description": "{street, city, province, postal_code, country}"},
            {"name": "home_type", "type": "ENUM", "constraints": "NOT NULL", "description": "single_family, condo, townhouse, apartment, multi_family, other"},
            {"name": "year_built", "type": "INTEGER", "constraints": "", "description": "Year of construction"},
            {"name": "total_square_footage", "type": "INTEGER", "constraints": "", "description": "Total living area in sq ft"},
            {"name": "lot_size", "type": "FLOAT", "constraints": "", "description": "Lot size in acres"},
            {"name": "num_bedrooms", "type": "INTEGER", "constraints": "", "description": "Total bedrooms"},
            {"name": "num_bathrooms", "type": "FLOAT", "constraints": "", "description": "Total bathrooms (0.5 for powder rooms)"},
            {"name": "num_floors", "type": "INTEGER", "constraints": "DEFAULT 1", "description": "Number of floors/levels"},
            {"name": "garage_spaces", "type": "INTEGER", "constraints": "", "description": "Number of garage parking spaces"},
            {"name": "has_basement", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "Has basement level"},
            {"name": "has_attic", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "Has attic level"},
            {"name": "digital_twin_completeness", "type": "FLOAT", "constraints": "DEFAULT 0.0", "description": "Completeness score 0.0-1.0"},
            {"name": "floor_plans_count", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Number of floor plans uploaded"},
            {"name": "rooms_count", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Total rooms detected/created"},
            {"name": "images_count", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Total images uploaded"},
            {"name": "last_analysis_date", "type": "TIMESTAMP", "constraints": "", "description": "Last AI analysis timestamp"},
            {"name": "extra_data", "type": "JSONB", "constraints": "DEFAULT '{}'", "description": "Additional metadata"},
            {"name": "created_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Creation timestamp"},
            {"name": "updated_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Last update timestamp"}
        ],
        "indexes": [
            "CREATE INDEX idx_homes_owner ON homes(owner_id)",
            "CREATE INDEX idx_homes_type ON homes(home_type)"
        ]
    },
    "floors": {
        "table_name": "floors",
        "description": "Individual floors/levels within a home",
        "columns": [
            {"name": "id", "type": "UUID", "constraints": "PRIMARY KEY", "description": "Unique identifier"},
            {"name": "home_id", "type": "UUID", "constraints": "FOREIGN KEY (homes.id) ON DELETE CASCADE", "description": "Parent home"},
            {"name": "floor_number", "type": "INTEGER", "constraints": "NOT NULL", "description": "0=basement, 1=main, 2=second, 99=attic"},
            {"name": "floor_name", "type": "VARCHAR(100)", "constraints": "NOT NULL", "description": "Display name (e.g., 'Main Floor', 'Basement')"},
            {"name": "floor_type", "type": "ENUM", "constraints": "", "description": "basement, main, upper, attic"},
            {"name": "total_area", "type": "FLOAT", "constraints": "", "description": "Total sq ft of this floor"},
            {"name": "ceiling_height", "type": "FLOAT", "constraints": "", "description": "Average ceiling height in feet"},
            {"name": "has_floor_plan", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "Floor plan uploaded?"},
            {"name": "rooms_count", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Number of rooms on this floor"},
            {"name": "extra_data", "type": "JSONB", "constraints": "DEFAULT '{}'", "description": "Additional metadata"},
            {"name": "created_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Creation timestamp"},
            {"name": "updated_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Last update timestamp"}
        ],
        "indexes": [
            "CREATE INDEX idx_floors_home ON floors(home_id)",
            "CREATE UNIQUE INDEX idx_floors_home_number ON floors(home_id, floor_number)"
        ]
    },
    "floor_plans": {
        "table_name": "floor_plans",
        "description": "Floor plan images and analysis",
        "columns": [
            {"name": "id", "type": "UUID", "constraints": "PRIMARY KEY", "description": "Unique identifier"},
            {"name": "home_id", "type": "UUID", "constraints": "FOREIGN KEY (homes.id) ON DELETE CASCADE", "description": "Parent home"},
            {"name": "floor_id", "type": "UUID", "constraints": "FOREIGN KEY (floors.id) ON DELETE CASCADE", "description": "Associated floor"},
            {"name": "name", "type": "VARCHAR(255)", "constraints": "", "description": "Floor plan name"},
            {"name": "floor_level", "type": "INTEGER", "constraints": "DEFAULT 1", "description": "Floor level (backward compatibility)"},
            {"name": "image_url", "type": "VARCHAR(500)", "constraints": "NOT NULL", "description": "Image file path/URL"},
            {"name": "image_width", "type": "INTEGER", "constraints": "", "description": "Image width in pixels"},
            {"name": "image_height", "type": "INTEGER", "constraints": "", "description": "Image height in pixels"},
            {"name": "image_format", "type": "VARCHAR(10)", "constraints": "", "description": "jpg, png, pdf, etc."},
            {"name": "file_size", "type": "INTEGER", "constraints": "", "description": "File size in bytes"},
            {"name": "scale", "type": "VARCHAR(100)", "constraints": "", "description": "Scale notation (e.g., '1/4 inch = 1 foot')"},
            {"name": "scale_ratio", "type": "FLOAT", "constraints": "", "description": "Numeric scale ratio"},
            {"name": "pixels_per_foot", "type": "FLOAT", "constraints": "", "description": "Pixels per foot for coordinate conversion"},
            {"name": "orientation", "type": "VARCHAR(20)", "constraints": "", "description": "north, south, east, west"},
            {"name": "is_analyzed", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "AI analysis completed?"},
            {"name": "analysis_date", "type": "TIMESTAMP", "constraints": "", "description": "Analysis timestamp"},
            {"name": "analysis_confidence", "type": "FLOAT", "constraints": "", "description": "Overall AI confidence 0.0-1.0"},
            {"name": "rooms_detected", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Number of rooms detected"},
            {"name": "total_area_detected", "type": "FLOAT", "constraints": "", "description": "Total area from AI analysis"},
            {"name": "analysis_metadata", "type": "JSONB", "constraints": "DEFAULT '{}'", "description": "Full AI analysis results"},
            {"name": "created_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Creation timestamp"},
            {"name": "updated_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Last update timestamp"}
        ],
        "indexes": [
            "CREATE INDEX idx_floor_plans_home ON floor_plans(home_id)",
            "CREATE INDEX idx_floor_plans_floor ON floor_plans(floor_id)",
            "CREATE INDEX idx_floor_plans_analyzed ON floor_plans(is_analyzed)"
        ]
    },
    "rooms": {
        "table_name": "rooms",
        "description": "Individual rooms within a home",
        "columns": [
            {"name": "id", "type": "UUID", "constraints": "PRIMARY KEY", "description": "Unique identifier"},
            {"name": "home_id", "type": "UUID", "constraints": "FOREIGN KEY (homes.id) ON DELETE CASCADE", "description": "Parent home"},
            {"name": "floor_id", "type": "UUID", "constraints": "FOREIGN KEY (floors.id) ON DELETE CASCADE", "description": "Parent floor"},
            {"name": "floor_plan_id", "type": "UUID", "constraints": "FOREIGN KEY (floor_plans.id)", "description": "Source floor plan"},
            {"name": "name", "type": "VARCHAR(255)", "constraints": "NOT NULL", "description": "Room name"},
            {"name": "room_type", "type": "ENUM", "constraints": "NOT NULL", "description": "75+ room types"},
            {"name": "floor_level", "type": "INTEGER", "constraints": "DEFAULT 1", "description": "Floor level (backward compatibility)"},
            {"name": "length", "type": "FLOAT", "constraints": "", "description": "Length in feet"},
            {"name": "width", "type": "FLOAT", "constraints": "", "description": "Width in feet"},
            {"name": "height", "type": "FLOAT", "constraints": "", "description": "Ceiling height in feet"},
            {"name": "area", "type": "FLOAT", "constraints": "", "description": "Floor area in sq ft"},
            {"name": "volume", "type": "FLOAT", "constraints": "", "description": "Volume in cubic feet"},
            {"name": "perimeter", "type": "FLOAT", "constraints": "", "description": "Perimeter in feet"},
            {"name": "position_x", "type": "FLOAT", "constraints": "", "description": "X coordinate on floor plan"},
            {"name": "position_y", "type": "FLOAT", "constraints": "", "description": "Y coordinate on floor plan"},
            {"name": "bounding_box", "type": "JSONB", "constraints": "", "description": "{x1, y1, x2, y2} on floor plan"},
            {"name": "polygon_coordinates", "type": "JSONB", "constraints": "", "description": "Precise room outline coordinates"},
            {"name": "condition_score", "type": "FLOAT", "constraints": "", "description": "Condition 0.0-1.0"},
            {"name": "style", "type": "VARCHAR(100)", "constraints": "", "description": "Design style"},
            {"name": "has_images", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "Has uploaded images?"},
            {"name": "image_count", "type": "INTEGER", "constraints": "DEFAULT 0", "description": "Number of images"},
            {"name": "is_analyzed", "type": "BOOLEAN", "constraints": "DEFAULT FALSE", "description": "AI analysis completed?"},
            {"name": "extra_data", "type": "JSONB", "constraints": "DEFAULT '{}'", "description": "Additional metadata"},
            {"name": "created_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Creation timestamp"},
            {"name": "updated_at", "type": "TIMESTAMP", "constraints": "DEFAULT NOW()", "description": "Last update timestamp"}
        ],
        "indexes": [
            "CREATE INDEX idx_rooms_home ON rooms(home_id)",
            "CREATE INDEX idx_rooms_floor ON rooms(floor_id)",
            "CREATE INDEX idx_rooms_floor_plan ON rooms(floor_plan_id)",
            "CREATE INDEX idx_rooms_type ON rooms(room_type)"
        ]
    }
}

print("✅ Complete schema defined")
print(f"\nTotal tables: {len(complete_schema)}")
for table_name, table_def in complete_schema.items():
    print(f"  - {table_name}: {len(table_def['columns'])} columns")

## 2. Generate Schema Documentation

In [ ]:
# Generate detailed documentation for each table
for table_name, table_def in complete_schema.items():
    print("\n" + "="*100)
    print(f"TABLE: {table_def['table_name'].upper()}")
    print("="*100)
    print(f"Description: {table_def['description']}")
    print("\nColumns:")
    print("-"*100)
    
    # Create DataFrame for better display
    df = pd.DataFrame(table_def['columns'])
    df = df[['name', 'type', 'constraints', 'description']]
    display(df)
    
    if 'indexes' in table_def:
        print("\nIndexes:")
        for idx in table_def['indexes']:
            print(f"  - {idx}")